# Question 1

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import copy
import json
import os

In [2]:
class ScaledDotProductAttention(nn.Module):
    def __init__(self):
        super().__init__()
        self.softmax = nn.Softmax(dim=-1)
        
    def forward(self, q, k, v, mask=None):
        d_k = k.size(-1)
        scores = torch.matmul(q, k.transpose(-2, -1)) / np.sqrt(d_k)
        if mask is not None:
            scores = scores.masked_fill(mask == 0, -1e9)
        attention = self.softmax(scores)
        output = torch.matmul(attention, v)
        return output, attention

class MultiHeadAttention(nn.Module):
    def __init__(self, embed_dim, n_heads):
        super().__init__()
        self.embed_dim = embed_dim
        self.n_heads = n_heads
        self.head_dim = embed_dim // n_heads
        
        self.q_linear = nn.Linear(embed_dim, embed_dim)
        self.k_linear = nn.Linear(embed_dim, embed_dim)
        self.v_linear = nn.Linear(embed_dim, embed_dim)
        
        self.attention = ScaledDotProductAttention()
        self.out_linear = nn.Linear(embed_dim, embed_dim)
        
    def forward(self, q, k, v, mask=None):
        batch_size = q.size(0)
        
        q = self.q_linear(q).view(batch_size, -1, self.n_heads, self.head_dim).transpose(1, 2)
        k = self.k_linear(k).view(batch_size, -1, self.n_heads, self.head_dim).transpose(1, 2)
        v = self.v_linear(v).view(batch_size, -1, self.n_heads, self.head_dim).transpose(1, 2)
        
        output, attention = self.attention(q, k, v, mask)
        
        output = output.transpose(1, 2).contiguous().view(batch_size, -1, self.embed_dim)
        output = self.out_linear(output)
        
        return output, attention

class MLPBlock(nn.Module):
    def __init__(self, embed_dim, mlp_dim, dropout=0.1):
        super().__init__()
        self.fc1 = nn.Linear(embed_dim, mlp_dim)
        self.fc2 = nn.Linear(mlp_dim, embed_dim)
        self.act = nn.GELU()
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        x = self.fc1(x)
        x = self.act(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.dropout(x)
        return x

class TransformerEncoderLayer(nn.Module):
    def __init__(self, embed_dim, n_heads, mlp_dim, dropout=0.1):
        super().__init__()
        self.attention = MultiHeadAttention(embed_dim, n_heads)
        self.norm1 = nn.LayerNorm(embed_dim)
        self.norm2 = nn.LayerNorm(embed_dim)
        self.mlp = MLPBlock(embed_dim, mlp_dim, dropout)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        attn_output, _ = self.attention(x, x, x)
        x = x + self.dropout(attn_output)
        x = self.norm1(x)
        
        mlp_output = self.mlp(x)
        x = x + self.dropout(mlp_output)
        x = self.norm2(x)
        
        return x


In [3]:
class VisionTransformer(nn.Module):
    def __init__(
        self,
        img_size=32,
        patch_size=4,
        in_channels=3,
        num_classes=10,
        embed_dim=256,
        depth=6,
        n_heads=8,
        mlp_dim=512,
        dropout=0.1,
    ):
        super().__init__()
        
        self.patch_size = patch_size
        self.num_patches = (img_size // patch_size) ** 2
        
        self.patch_embed = nn.Conv2d(
            in_channels, embed_dim, 
            kernel_size=patch_size, stride=patch_size
        )
      
        self.cls_token = nn.Parameter(torch.zeros(1, 1, embed_dim))
        
        self.pos_embed = nn.Parameter(torch.zeros(1, self.num_patches + 1, embed_dim))
        
        self.dropout = nn.Dropout(dropout)
        
        self.transformer_encoder = nn.ModuleList([
            TransformerEncoderLayer(embed_dim, n_heads, mlp_dim, dropout)
            for _ in range(depth)
        ])
        
        self.norm = nn.LayerNorm(embed_dim)
        self.head = nn.Linear(embed_dim, num_classes)

        self._init_weights()
        
    def _init_weights(self):
        nn.init.normal_(self.cls_token, std=0.02)
        nn.init.normal_(self.pos_embed, std=0.02)
        self.apply(self._init_linear_weights)
            
    def _init_linear_weights(self, m):
        if isinstance(m, nn.Linear):
            nn.init.normal_(m.weight, std=0.02)
            if m.bias is not None:
                nn.init.zeros_(m.bias)
        elif isinstance(m, nn.LayerNorm):
            nn.init.zeros_(m.bias)
            nn.init.ones_(m.weight)
        
    def forward(self, x):
        batch_size = x.size(0)
        
        x = self.patch_embed(x)
       
        x = x.flatten(2)
        x = x.transpose(1, 2)
      
        cls_tokens = self.cls_token.expand(batch_size, -1, -1)
        x = torch.cat((cls_tokens, x), dim=1)
        
        x = x + self.pos_embed
        x = self.dropout(x)
        
        for layer in self.transformer_encoder:
            x = layer(x)
        
        x = self.norm(x)
        x = x[:, 0]
        
        x = self.head(x)
        
        return x

In [4]:
def train_epoch(model, train_loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    train_pbar = tqdm(train_loader, desc="Training")
    
    for images, labels in train_pbar:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
        
        train_pbar.set_postfix({
            'loss': running_loss / (train_pbar.n + 1),
            'acc': 100. * correct / total
        })
    
    train_loss = running_loss / len(train_loader)
    train_acc = 100. * correct / total
    
    return train_loss, train_acc

def evaluate(model, test_loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
    
    test_loss = running_loss / len(test_loader)
    test_acc = 100. * correct / total
    
    return test_loss, test_acc

In [5]:
def get_data_loaders(batch_size=128, augmentation='default'):
    if augmentation == 'default':
        transform_train = transforms.Compose([
            transforms.RandomCrop(32, padding=4),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616)),
        ])
    elif augmentation == 'strong':
        transform_train = transforms.Compose([
            transforms.RandomCrop(32, padding=4),
            transforms.RandomHorizontalFlip(),
            transforms.RandomRotation(15),
            transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
            transforms.ToTensor(),
            transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616)),
            transforms.RandomErasing(p=0.5, scale=(0.02, 0.33), ratio=(0.3, 3.3)),
        ])
    elif augmentation == 'autoaugment':
        transform_train = transforms.Compose([
            transforms.RandomCrop(32, padding=4),
            transforms.RandomHorizontalFlip(),
            transforms.AutoAugment(transforms.AutoAugmentPolicy.CIFAR10),
            transforms.ToTensor(),
            transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616)),
        ])
    elif augmentation == 'cutmix':
        transform_train = transforms.Compose([
            transforms.RandomCrop(32, padding=4),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616)),
        ])
    else:
        raise ValueError(f"Unknown augmentation: {augmentation}")
    
    transform_test = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616)),
    ])
    
    train_dataset = torchvision.datasets.CIFAR10(
        root='./data', train=True, download=True, transform=transform_train)
    
    test_dataset = torchvision.datasets.CIFAR10(
        root='./data', train=False, download=True, transform=transform_test)
    
    train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
    
    test_loader = torch.utils.data.DataLoader(
        test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)
    
    return train_loader, test_loader

In [6]:
def train_model(config, device, num_epochs=20):
    patch_size = config['patch_size']
    embed_dim = config['embed_dim']
    depth = config['depth']
    n_heads = config['n_heads']
    mlp_dim = config['mlp_dim']
    batch_size = config['batch_size']
    lr = config['lr']
    weight_decay = config['weight_decay']
    augmentation = config.get('augmentation', 'default')
   
    train_loader, test_loader = get_data_loaders(batch_size, augmentation)
    
    model = VisionTransformer(
        img_size=32,
        patch_size=patch_size,
        in_channels=3,
        num_classes=10,
        embed_dim=embed_dim,
        depth=depth,
        n_heads=n_heads,
        mlp_dim=mlp_dim,
        dropout=0.1
    ).to(device)
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs)
    
    train_losses, train_accs = [], []
    test_losses, test_accs = [], []
    best_acc = 0.0
    best_model_state = None
    
    for epoch in range(num_epochs):
        print(f"\nEpoch {epoch+1}/{num_epochs}")
        
        train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device)
        test_loss, test_acc = evaluate(model, test_loader, criterion, device)
        
        scheduler.step()
        
        train_losses.append(train_loss)
        train_accs.append(train_acc)
        test_losses.append(test_loss)
        test_accs.append(test_acc)
        
        print(f'Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%')
        print(f'Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.2f}%')
        
        if test_acc > best_acc:
            best_acc = test_acc
            best_model_state = copy.deepcopy(model.state_dict())
    
    if best_model_state is not None:
        model.load_state_dict(best_model_state)
    
    results = {
        'config': config,
        'train_losses': train_losses,
        'train_accs': train_accs,
        'test_losses': test_losses,
        'test_accs': test_accs,
        'best_test_acc': best_acc
    }
    
    exp_name = f"vit_ps{patch_size}_e{embed_dim}_d{depth}_h{n_heads}_mlp{mlp_dim}"
    if augmentation != 'default':
        exp_name += f"_aug{augmentation}"
    
    if not os.path.exists('results'):
        os.makedirs('results')
    
    torch.save(model.state_dict(), f'results/{exp_name}_model.pth')
    with open(f'results/{exp_name}_results.json', 'w') as f:
        json.dump({k: v if not isinstance(v, list) else [float(i) for i in v] 
                  for k, v in results.items() if k != 'config'}, f)
    
    return model, results


In [7]:
def plot_comparison_results(all_results, comparison_name, filename):
    plt.figure(figsize=(15, 10))
    
    keys = list(all_results.keys())
    
    plt.subplot(2, 2, 1)
    for key in keys:
        plt.plot(all_results[key]['train_losses'], label=f'{comparison_name} = {key}')
    plt.xlabel('Epoch')
    plt.ylabel('Training Loss')
    plt.legend()
    plt.title('Training Loss vs. Epoch')
    
    plt.subplot(2, 2, 2)
    for key in keys:
        plt.plot(all_results[key]['test_losses'], label=f'{comparison_name} = {key}')
    plt.xlabel('Epoch')
    plt.ylabel('Testing Loss')
    plt.legend()
    plt.title('Testing Loss vs. Epoch')
    
    plt.subplot(2, 2, 3)
    for key in keys:
        plt.plot(all_results[key]['train_accs'], label=f'{comparison_name} = {key}')
    plt.xlabel('Epoch')
    plt.ylabel('Training Accuracy (%)')
    plt.legend()
    plt.title('Training Accuracy vs. Epoch')
    
    plt.subplot(2, 2, 4)
    for key in keys:
        plt.plot(all_results[key]['test_accs'], label=f'{comparison_name} = {key}')
    plt.xlabel('Epoch')
    plt.ylabel('Testing Accuracy (%)')
    plt.legend()
    plt.title('Testing Accuracy vs. Epoch')
    
    plt.tight_layout()
    plt.savefig(f'results/{filename}.png')
    plt.close()

def plot_augmentation_results(all_results, comparison_name, filename):
    plt.figure(figsize=(15, 5))
    
    keys = list(all_results.keys())
    
    plt.subplot(1, 2, 1)
    for key in keys:
        plt.plot(all_results[key]['test_accs'], label=f'{key}')
    plt.xlabel('Epoch')
    plt.ylabel('Testing Accuracy (%)')
    plt.legend()
    plt.title('Testing Accuracy vs. Epoch')
    
    plt.subplot(1, 2, 2)
    best_accs = [all_results[key]['best_test_acc'] for key in keys]
    plt.bar(keys, best_accs)
    plt.xlabel('Augmentation Type')
    plt.ylabel('Best Test Accuracy (%)')
    plt.title('Best Test Accuracy by Augmentation')
    
    plt.tight_layout()
    plt.savefig(f'results/{filename}.png')
    plt.close()


In [8]:
def run_patch_size_experiment(device, num_epochs=15):
    patch_sizes = [2, 4, 8]
    all_results = {}
    
    base_config = {
        'batch_size': 128,
        'lr': 1e-3,
        'weight_decay': 0.05,
        'augmentation': 'default',
    }
    
    for patch_size in patch_sizes:
        print(f"\n=== Running experiment with patch_size={patch_size} ===")
        
        if patch_size == 2:
            embed_dim = 192
            mlp_dim = 384
        elif patch_size == 4:
            embed_dim = 256
            mlp_dim = 512
        else:  # patch_size == 8
            embed_dim = 384
            mlp_dim = 768
        
        config = {**base_config, 
                  'patch_size': patch_size,
                  'embed_dim': embed_dim,
                  'depth': 6,
                  'n_heads': 8,
                  'mlp_dim': mlp_dim}
        
        _, results = train_model(config, device, num_epochs)
        all_results[patch_size] = results
    
    plot_comparison_results(all_results, "Patch Size", "vit_patch_size_comparison")
    
    print("\nFinal Best Test Accuracies:")
    for patch_size in patch_sizes:
        print(f"Patch Size = {patch_size}: {all_results[patch_size]['best_test_acc']:.2f}%")
    
    return all_results

def run_hyperparameter_exploration(device, num_epochs=50):
    configs = [
        # Base configuration
        {
            'patch_size': 4,
            'embed_dim': 256,
            'depth': 6,
            'n_heads': 8,  
            'mlp_dim': 512,
            'batch_size': 128,
            'lr': 1e-3,
            'weight_decay': 0.05,
            'augmentation': 'default',
        },
        # Variation 1: Deeper model
        {
            'patch_size': 4,
            'embed_dim': 256,
            'depth': 8,
            'n_heads': 8,  
            'mlp_dim': 512,
            'batch_size': 128,
            'lr': 1e-3,
            'weight_decay': 0.05,
            'augmentation': 'default',
        },
        # Variation 2: Wider model
        {
            'patch_size': 4,
            'embed_dim': 384,
            'depth': 6,
            'n_heads': 12,  
            'mlp_dim': 768,
            'batch_size': 128,
            'lr': 1e-3,
            'weight_decay': 0.05,
            'augmentation': 'default',
        },
        # Variation 3: More attention heads - FIXED
        {
            'patch_size': 4,
            'embed_dim': 288,  
            'depth': 6,
            'n_heads': 12,
            'mlp_dim': 576,    
            'batch_size': 128,
            'lr': 1e-3,
            'weight_decay': 0.05,
            'augmentation': 'default',
        },
        # Variation 4: Different learning rate
        {
            'patch_size': 4,
            'embed_dim': 256,
            'depth': 6,
            'n_heads': 8,  
            'mlp_dim': 512,
            'batch_size': 128,
            'lr': 5e-4,
            'weight_decay': 0.05,
            'augmentation': 'default',
        },
    ]
    
    all_results = []
    
    for i, config in enumerate(configs):
        print(f"\n=== Running hyperparameter configuration {i+1}/{len(configs)} ===")
        
        assert config['embed_dim'] % config['n_heads'] == 0, \
            f"Configuration {i+1}: embed_dim ({config['embed_dim']}) must be divisible by n_heads ({config['n_heads']})"
        
        _, results = train_model(config, device, num_epochs)
        all_results.append(results)
    
    best_idx = max(range(len(all_results)), key=lambda i: all_results[i]['best_test_acc'])
    best_config = configs[best_idx]
    best_acc = all_results[best_idx]['best_test_acc']
    
    return best_config, all_results

In [9]:
def run_augmentation_experiment(device, best_config, num_epochs=20):
    augmentations = ['default', 'strong', 'autoaugment', 'cutmix']
    all_results = {}
    
    for aug in augmentations:
        print(f"\n=== Running experiment with augmentation={aug} ===")
        
        config = {**best_config, 'augmentation': aug}
        
        _, results = train_model(config, device, num_epochs)
        all_results[aug] = results
    
    plot_augmentation_results(all_results, "Data Augmentation", "vit_augmentation_comparison")
    
    print("\nFinal Best Test Accuracies with Different Augmentations:")
    for aug in augmentations:
        print(f"{aug}: {all_results[aug]['best_test_acc']:.2f}%")
    
    best_aug = max(augmentations, key=lambda aug: all_results[aug]['best_test_acc'])
    best_aug_acc = all_results[best_aug]['best_test_acc']
    
    print(f"\nBest Augmentation: {best_aug} (Test Acc: {best_aug_acc:.2f}%)")
    
    return all_results


In [23]:
torch.manual_seed(42)
np.random.seed(42)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

if not os.path.exists('results'):
    os.makedirs('results')

experiments_to_run = ['patch_size', 'hyperparameters', 'augmentation']

results = {}

if 'patch_size' in experiments_to_run:
    print("\n=== Running Patch Size Variation Experiment ===")
    results['patch_size'] = run_patch_size_experiment(device)

Using device: cuda

=== Running Patch Size Variation Experiment ===

=== Running experiment with patch_size=2 ===

Epoch 1/15


Training: 100%|██████████| 391/391 [00:54<00:00,  7.16it/s, loss=1.73, acc=35.3]


Train Loss: 1.7289, Train Acc: 35.34%
Test Loss: 1.5368, Test Acc: 46.10%

Epoch 2/15


Training: 100%|██████████| 391/391 [00:55<00:00,  7.10it/s, loss=1.32, acc=52.1]


Train Loss: 1.3186, Train Acc: 52.07%
Test Loss: 1.2039, Test Acc: 56.28%

Epoch 3/15


...

Train Loss: 2.3027, Train Acc: 9.84%
Test Loss: 2.3026, Test Acc: 10.00%

Epoch 14/15


Training: 100%|██████████| 391/391 [00:13<00:00, 29.24it/s, loss=2.31, acc=9.79]


Train Loss: 2.3027, Train Acc: 9.79%
Test Loss: 2.3026, Test Acc: 10.00%

Epoch 15/15


Training: 100%|██████████| 391/391 [00:12<00:00, 30.38it/s, loss=2.32, acc=10]  


Train Loss: 2.3027, Train Acc: 10.03%
Test Loss: 2.3026, Test Acc: 10.00%

Final Best Test Accuracies:
Patch Size = 2: 78.89%
Patch Size = 4: 78.71%
Patch Size = 8: 12.76%


Patch size 8 performs terribly (~10% accuracy) because it divides 32×32 CIFAR-10 images into only 4×4=16 patches, creating an extreme information bottleneck that loses critical details needed for classification. In contrast, patch sizes 2 and 4 work well because they preserve sufficient spatial resolution while providing enough tokens for the attention mechanism to learn meaningful relationships. The results confirm that appropriate patch size selection is critical relative to input image dimensions.

In [35]:
if 'hyperparameters' in experiments_to_run:
    print("\n=== Running Hyperparameter Exploration ===")
    best_config, hyperparameter_results = run_hyperparameter_exploration(device)
    results['hyperparameters'] = hyperparameter_results


=== Running Hyperparameter Exploration ===

=== Running hyperparameter configuration 1/5 ===

Epoch 1/50


Training: 100%|██████████| 391/391 [00:14<00:00, 27.03it/s, loss=1.77, acc=34.2]


Train Loss: 1.7611, Train Acc: 34.20%
Test Loss: 1.4996, Test Acc: 44.66%

Epoch 2/50


Training: 100%|██████████| 391/391 [00:14<00:00, 26.83it/s, loss=1.43, acc=47.9]


Train Loss: 1.4223, Train Acc: 47.87%
Test Loss: 1.2701, Test Acc: 53.64%

Epoch 3/50


...

Train Loss: 0.1704, Train Acc: 93.92%
Test Loss: 0.5465, Test Acc: 84.56%

Epoch 49/50


Training: 100%|██████████| 391/391 [00:14<00:00, 26.14it/s, loss=0.174, acc=93.8]


Train Loss: 0.1738, Train Acc: 93.80%
Test Loss: 0.5442, Test Acc: 84.61%

Epoch 50/50


Training: 100%|██████████| 391/391 [00:14<00:00, 26.10it/s, loss=0.17, acc=94.1] 


Train Loss: 0.1693, Train Acc: 94.06%
Test Loss: 0.5449, Test Acc: 84.56%

Hyperparameter Exploration Results:
Config 1:
  Patch Size: 4
  Embed Dim: 256
  Depth: 6
  Heads: 8
  MLP Dim: 512
  Learning Rate: 0.001
  Best Test Accuracy: 83.81%
Config 2:
  Patch Size: 4
  Embed Dim: 256
  Depth: 8
  Heads: 8
  MLP Dim: 512
  Learning Rate: 0.001
  Best Test Accuracy: 83.99%
Config 3:
  Patch Size: 4
  Embed Dim: 384
  Depth: 6
  Heads: 12
  MLP Dim: 768
  Learning Rate: 0.001
  Best Test Accuracy: 10.04%
Config 4:
  Patch Size: 4
  Embed Dim: 288
  Depth: 6
  Heads: 12
  MLP Dim: 576
  Learning Rate: 0.001
  Best Test Accuracy: 83.38%
Config 5:
  Patch Size: 4
  Embed Dim: 256
  Depth: 6
  Heads: 8
  MLP Dim: 512
  Learning Rate: 0.0005
  Best Test Accuracy: 84.61%

Best Configuration (Test Acc: 84.61%):
  Patch Size: 4
  Embed Dim: 256
  Depth: 6
  Heads: 8
  MLP Dim: 512
  Learning Rate: 0.0005


In [ ]:
all_results = results['hyperparameters']

In [40]:
print("\nHyperparameter Exploration Results:")
print("=" * 50)
for i, results in enumerate(all_results):
    config = results["config"]
    print(f"Config {i+1}:")
    print(f"  Patch Size: {config['patch_size']}")
    print(f"  Embed Dim: {config['embed_dim']}")
    print(f"  Depth: {config['depth']}")
    print(f"  Heads: {config['n_heads']}")
    print(f"  MLP Dim: {config['mlp_dim']}")
    print(f"  Learning Rate: {config['lr']}")
    print(f"  Best Test Accuracy: {results['best_test_acc']:.2f}%")
    print("=" * 50)

# Find best configuration
best_idx = max(range(len(all_results)), key=lambda i: all_results[i]['best_test_acc'])
best_config = all_results[best_idx]['config']
best_acc = all_results[best_idx]['best_test_acc']

print(f"\nBest Configuration (Test Acc: {best_acc:.2f}%):")
print(f"  Patch Size: {best_config['patch_size']}")
print(f"  Embed Dim: {best_config['embed_dim']}")
print(f"  Depth: {best_config['depth']}")
print(f"  Heads: {best_config['n_heads']}")
print(f"  MLP Dim: {best_config['mlp_dim']}")
print(f"  Learning Rate: {best_config['lr']}")


Hyperparameter Exploration Results:
Config 1:
  Patch Size: 4
  Embed Dim: 256
  Depth: 6
  Heads: 8
  MLP Dim: 512
  Learning Rate: 0.001
  Best Test Accuracy: 83.81%
Config 2:
  Patch Size: 4
  Embed Dim: 256
  Depth: 8
  Heads: 8
  MLP Dim: 512
  Learning Rate: 0.001
  Best Test Accuracy: 83.99%
Config 3:
  Patch Size: 4
  Embed Dim: 384
  Depth: 6
  Heads: 12
  MLP Dim: 768
  Learning Rate: 0.001
  Best Test Accuracy: 10.04%
Config 4:
  Patch Size: 4
  Embed Dim: 288
  Depth: 6
  Heads: 12
  MLP Dim: 576
  Learning Rate: 0.001
  Best Test Accuracy: 83.38%
Config 5:
  Patch Size: 4
  Embed Dim: 256
  Depth: 6
  Heads: 8
  MLP Dim: 512
  Learning Rate: 0.0005
  Best Test Accuracy: 84.61%

Best Configuration (Test Acc: 84.61%):
  Patch Size: 4
  Embed Dim: 256
  Depth: 6
  Heads: 8
  MLP Dim: 512
  Learning Rate: 0.0005


The hyperparameter exploration results reveal that configuration 5 performs best (84.61% accuracy) with a lower learning rate (0.0005) compared to other setups. Interestingly, configuration 3 fails completely (10.04% accuracy) despite having the largest embedding dimension (384) and MLP size (768), likely because the model becomes too complex and struggles to converge properly. The optimal configuration balances model capacity (256 embedding dimension, 6 transformer layers, 8 attention heads) with training stability. The results show that bigger isn't always better - carefully tuned smaller models often outperform larger ones, and learning rate has a significant impact on final performance.

In [36]:
if 'augmentation' in experiments_to_run:
    print("\n=== Running Data Augmentation Experiment ===")
    results['augmentation'] = run_augmentation_experiment(device, best_config)


=== Running Data Augmentation Experiment ===

=== Running experiment with augmentation=default ===

Epoch 1/20


Training: 100%|██████████| 391/391 [00:14<00:00, 26.64it/s, loss=1.7, acc=37.3] 


Train Loss: 1.6909, Train Acc: 37.30%
Test Loss: 1.3674, Test Acc: 50.27%

Epoch 2/20


Training: 100%|██████████| 391/391 [00:14<00:00, 26.44it/s, loss=1.32, acc=52.4]


Train Loss: 1.3126, Train Acc: 52.41%
Test Loss: 1.2181, Test Acc: 56.46%

Epoch 3/20


...

Train Loss: 0.5181, Train Acc: 81.60%
Test Loss: 0.6179, Test Acc: 78.98%

Epoch 19/20


Training: 100%|██████████| 391/391 [00:14<00:00, 26.59it/s, loss=0.51, acc=81.9] 


Train Loss: 0.5073, Train Acc: 81.92%
Test Loss: 0.6086, Test Acc: 79.35%

Epoch 20/20


Training: 100%|██████████| 391/391 [00:14<00:00, 26.51it/s, loss=0.505, acc=82.1]


Train Loss: 0.5027, Train Acc: 82.11%
Test Loss: 0.6100, Test Acc: 79.30%

Final Best Test Accuracies with Different Augmentations:
default: 79.56%
strong: 76.40%
autoaugment: 76.92%
cutmix: 79.35%

Best Augmentation: default (Test Acc: 79.56%)


the default augmentation (simple cropping and flipping) outperforms more complex augmentation techniques on CIFAR-10. This suggests that Vision Transformers may be sensitive to excessive perturbations in small-scale datasets. Strong augmentation techniques like random rotations, color jittering, and random erasing actually hurt performance by creating training samples that deviate too far from the test distribution. AutoAugment, despite being optimized for CIFAR-10, also underperforms likely because its policy was developed for CNNs rather than transformer architectures. CutMix performs nearly as well as the default because it preserves more original image information while still providing regularization benefits. The results indicate that for Vision Transformers on small images, simpler augmentation strategies that maintain image integrity work best.

### Position Embedding

In [9]:
import math 

class VisionTransformerWithPosEmbed(nn.Module):
    def __init__(
        self,
        img_size=32,
        patch_size=4,
        in_channels=3,
        num_classes=10,
        embed_dim=256,
        depth=6,
        n_heads=8,
        mlp_dim=512,
        dropout=0.1,
        pos_embedding_type='learned_1d'  
    ):
        super().__init__()
        
        self.patch_size = patch_size
        self.num_patches = (img_size // patch_size) ** 2
        self.pos_embedding_type = pos_embedding_type
        
        self.patch_embed = nn.Conv2d(
            in_channels, embed_dim, 
            kernel_size=patch_size, stride=patch_size
        )
        
        self.cls_token = nn.Parameter(torch.zeros(1, 1, embed_dim))
        
        if pos_embedding_type == 'none':
            self.pos_embed = None
        elif pos_embedding_type == 'learned_1d':
            self.pos_embed = nn.Parameter(torch.zeros(1, self.num_patches + 1, embed_dim))
        elif pos_embedding_type == 'learned_2d':
            h_patches = w_patches = img_size // patch_size
            self.pos_embed_x = nn.Parameter(torch.zeros(1, h_patches, embed_dim // 2))
            self.pos_embed_y = nn.Parameter(torch.zeros(1, w_patches, embed_dim // 2))
        elif pos_embedding_type == 'sinusoidal':
            self.register_buffer('pos_embed', self._create_sinusoidal_embeddings(
                self.num_patches + 1, embed_dim
            ))
        else:
            raise ValueError(f"Unknown positional embedding type: {pos_embedding_type}")
        
        self.dropout = nn.Dropout(dropout)
        
        self.transformer_encoder = nn.ModuleList([
            TransformerEncoderLayer(embed_dim, n_heads, mlp_dim, dropout)
            for _ in range(depth)
        ])
        
        self.norm = nn.LayerNorm(embed_dim)
        self.head = nn.Linear(embed_dim, num_classes)

        self._init_weights()
    
    def _init_weights(self):
        nn.init.normal_(self.cls_token, std=0.02)
        if self.pos_embedding_type == 'learned_1d':
            nn.init.normal_(self.pos_embed, std=0.02)
        elif self.pos_embedding_type == 'learned_2d':
            nn.init.normal_(self.pos_embed_x, std=0.02)
            nn.init.normal_(self.pos_embed_y, std=0.02)
        
        self.apply(self._init_linear_weights)
    
    def _init_linear_weights(self, m):
        if isinstance(m, nn.Linear):
            nn.init.normal_(m.weight, std=0.02)
            if m.bias is not None:
                nn.init.zeros_(m.bias)
        elif isinstance(m, nn.LayerNorm):
            nn.init.zeros_(m.bias)
            nn.init.ones_(m.weight)
    
    def _create_sinusoidal_embeddings(self, seq_len, embed_dim):
        position = torch.arange(seq_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, embed_dim, 2) * (-math.log(10000.0) / embed_dim))
        
        pos_embedding = torch.zeros(1, seq_len, embed_dim)
        pos_embedding[0, :, 0::2] = torch.sin(position * div_term)
        pos_embedding[0, :, 1::2] = torch.cos(position * div_term)
        
        return pos_embedding
    
    def _generate_2d_pos_embed(self, batch_size):
        h_patches = w_patches = int(math.sqrt(self.num_patches))
        
        pos_embed_x = self.pos_embed_x.expand(batch_size, -1, -1)
        pos_embed_y = self.pos_embed_y.expand(batch_size, -1, -1)
        
        x_emb = pos_embed_x.unsqueeze(2).repeat(1, 1, w_patches, 1)
        y_emb = pos_embed_y.unsqueeze(1).repeat(1, h_patches, 1, 1)
        
        pos_2d = torch.cat([x_emb, y_emb], dim=-1)
        pos_2d = pos_2d.reshape(batch_size, self.num_patches, -1)
        
        cls_pos = torch.zeros(batch_size, 1, pos_2d.size(-1), device=pos_2d.device)
        pos_embedding = torch.cat([cls_pos, pos_2d], dim=1)
        
        return pos_embedding
    
    def forward(self, x):
        batch_size = x.size(0)
        
        x = self.patch_embed(x)
        
        x = x.flatten(2)
        x = x.transpose(1, 2)
        
        cls_tokens = self.cls_token.expand(batch_size, -1, -1)
        x = torch.cat((cls_tokens, x), dim=1)
        
        if self.pos_embedding_type == 'none':
            pass
        elif self.pos_embedding_type == 'learned_1d':
            x = x + self.pos_embed
        elif self.pos_embedding_type == 'learned_2d':
            pos_embedding = self._generate_2d_pos_embed(batch_size)
            x = x + pos_embedding
        elif self.pos_embedding_type == 'sinusoidal':
            x = x + self.pos_embed
        
        x = self.dropout(x)
        for layer in self.transformer_encoder:
            x = layer(x)
        x = self.norm(x)
        x = x[:, 0]
        x = self.head(x)
        
        return x


In [10]:
def train_with_pos_embedding(pos_embedding_type, device, best_config, num_epochs=15):
    print(f"\n=== Training with {pos_embedding_type} positional embedding ===")
    
    batch_size = best_config['batch_size']
    
    transform_train = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616)),
    ])
    
    transform_test = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616)),
    ])
    
    train_dataset = torchvision.datasets.CIFAR10(
        root='./data', train=True, download=True, transform=transform_train)
    
    test_dataset = torchvision.datasets.CIFAR10(
        root='./data', train=False, download=True, transform=transform_test)
    
    train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
    
    test_loader = torch.utils.data.DataLoader(
        test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)
    
    model = VisionTransformerWithPosEmbed(
        img_size=32,
        patch_size=best_config['patch_size'],
        in_channels=3,
        num_classes=10,
        embed_dim=best_config['embed_dim'],
        depth=best_config['depth'],
        n_heads=best_config['n_heads'],
        mlp_dim=best_config['mlp_dim'],
        dropout=0.1,
        pos_embedding_type=pos_embedding_type
    ).to(device)
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(model.parameters(), lr=best_config['lr'], weight_decay=best_config['weight_decay'])
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs)
    
    train_losses, train_accs = [], []
    test_losses, test_accs = [], []
    best_test_acc = 0.0
    best_model_state = None
    
    for epoch in range(num_epochs):
        print(f"\nEpoch {epoch+1}/{num_epochs}")
        
        train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device)
        test_loss, test_acc = evaluate(model, test_loader, criterion, device)
        
        scheduler.step()
        
        train_losses.append(train_loss)
        train_accs.append(train_acc)
        test_losses.append(test_loss)
        test_accs.append(test_acc)
        
        print(f'Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%')
        print(f'Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.2f}%')
        
        if test_acc > best_test_acc:
            best_test_acc = test_acc
            best_model_state = copy.deepcopy(model.state_dict())
    
    if best_model_state is not None:
        model.load_state_dict(best_model_state)
    
    if not os.path.exists('saved_models'):
        os.makedirs('saved_models')
    
    model_path = f'saved_models/vit_cifar10_{pos_embedding_type}_best.pth'
    torch.save({
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'scheduler_state_dict': scheduler.state_dict(),
        'best_test_acc': best_test_acc,
        'config': best_config,
        'pos_embedding_type': pos_embedding_type
    }, model_path)
    
    print(f"Model saved to {model_path}")
    
    results = {
        'pos_embedding_type': pos_embedding_type,
        'train_losses': train_losses,
        'train_accs': train_accs,
        'test_losses': test_losses,
        'test_accs': test_accs,
        'final_test_acc': test_accs[-1],
        'best_test_acc': best_test_acc,
        'model_path': model_path
    }
    
    return results, model

def run_positional_embedding_experiment(device, best_config):
    pos_embedding_types = ['none', 'learned_1d', 'learned_2d', 'sinusoidal']
    all_results = {}
    all_models = {}  
    
    for pe_type in pos_embedding_types:
        results, model = train_with_pos_embedding(pe_type, device, best_config)
        all_results[pe_type] = results
        all_models[pe_type] = model
    
    plt.figure(figsize=(15, 10))
    
    plt.subplot(2, 2, 1)
    for pe_type in pos_embedding_types:
        plt.plot(all_results[pe_type]['train_losses'], label=f'{pe_type}')
    plt.xlabel('Epoch')
    plt.ylabel('Training Loss')
    plt.legend()
    plt.title('Training Loss vs. Epoch')
    
    plt.subplot(2, 2, 2)
    for pe_type in pos_embedding_types:
        plt.plot(all_results[pe_type]['test_losses'], label=f'{pe_type}')
    plt.xlabel('Epoch')
    plt.ylabel('Testing Loss')
    plt.legend()
    plt.title('Testing Loss vs. Epoch')
    
    plt.subplot(2, 2, 3)
    for pe_type in pos_embedding_types:
        plt.plot(all_results[pe_type]['train_accs'], label=f'{pe_type}')
    plt.xlabel('Epoch')
    plt.ylabel('Training Accuracy (%)')
    plt.legend()
    plt.title('Training Accuracy vs. Epoch')
    
    plt.subplot(2, 2, 4)
    for pe_type in pos_embedding_types:
        plt.plot(all_results[pe_type]['test_accs'], label=f'{pe_type}')
    plt.xlabel('Epoch')
    plt.ylabel('Testing Accuracy (%)')
    plt.legend()
    plt.title('Testing Accuracy vs. Epoch')
    
    plt.tight_layout()
    plt.savefig('vit_positional_embedding_comparison.png')
    plt.close()
    
    plt.figure(figsize=(10, 6))
    final_accs = [all_results[pe_type]['best_test_acc'] for pe_type in pos_embedding_types]
    plt.bar(pos_embedding_types, final_accs)
    plt.xlabel('Positional Embedding Type')
    plt.ylabel('Test Accuracy (%)')
    plt.title('Best Test Accuracy by Positional Embedding Type')
    for i, acc in enumerate(final_accs):
        plt.text(i, acc + 0.5, f'{acc:.2f}%', ha='center')
    plt.ylim(0, 100)
    plt.savefig('vit_positional_embedding_final_accuracy.png')
    plt.close()
    
    print("\nBest Test Accuracies by Positional Embedding Type:")
    for pe_type in pos_embedding_types:
        print(f"{pe_type}: {all_results[pe_type]['best_test_acc']:.2f}%")
        print(f"   Model saved at: {all_results[pe_type]['model_path']}")
    
    return all_results, all_models

In [12]:
torch.manual_seed(42)
np.random.seed(42)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

best_config = {
    'patch_size': 4,
    'embed_dim': 256,
    'depth': 6,
    'n_heads': 8,
    'mlp_dim': 512,
    'batch_size': 128,
    'lr': 0.0005,
    'weight_decay': 0.05,
}

results = run_positional_embedding_experiment(device, best_config)

Using device: cuda

=== Training with none positional embedding ===

Epoch 1/15


Training: 100%|██████████| 391/391 [00:16<00:00, 23.09it/s, loss=1.73, acc=35.6]


Train Loss: 1.7218, Train Acc: 35.62%
Test Loss: 1.4960, Test Acc: 46.14%

Epoch 2/15


Training: 100%|██████████| 391/391 [00:14<00:00, 26.49it/s, loss=1.39, acc=50]  


Train Loss: 1.3806, Train Acc: 50.05%
Test Loss: 1.3152, Test Acc: 52.98%

Epoch 3/15


...

Train Loss: 0.6730, Train Acc: 75.92%
Test Loss: 0.7087, Test Acc: 75.27%

Epoch 14/15


Training: 100%|██████████| 391/391 [00:15<00:00, 25.95it/s, loss=0.656, acc=76.9]


Train Loss: 0.6524, Train Acc: 76.85%
Test Loss: 0.6977, Test Acc: 75.62%

Epoch 15/15


Training: 100%|██████████| 391/391 [00:15<00:00, 25.58it/s, loss=0.645, acc=77.3]


Train Loss: 0.6417, Train Acc: 77.32%
Test Loss: 0.6944, Test Acc: 75.85%
Model saved to saved_models/vit_cifar10_sinusoidal_best.pth

Best Test Accuracies by Positional Embedding Type:
none: 69.60%
   Model saved at: saved_models/vit_cifar10_none_best.pth
learned_1d: 77.57%
   Model saved at: saved_models/vit_cifar10_learned_1d_best.pth
learned_2d: 78.64%
   Model saved at: saved_models/vit_cifar10_learned_2d_best.pth
sinusoidal: 75.85%
   Model saved at: saved_models/vit_cifar10_sinusoidal_best.pth


The positional embedding comparison shows learned representations (especially 2D at 78.64%) significantly outperform fixed patterns (sinusoidal at 75.85%) and no position information (69.60%). This makes sense since learned embeddings can adapt to the specific spatial relationships in CIFAR-10 images, while the model without position embeddings struggles to understand spatial structure completely.
For augmentations, the graph shows all methods eventually converge to similar accuracies, with default and cutmix slightly outperforming others. This suggests Vision Transformers are somewhat robust to augmentation choice on CIFAR-10, but simpler augmentations that maintain original image structure lead to more stable learning curves and marginally better performance.

In [11]:
class VisionTransformerForVisualization(nn.Module):
    def __init__(self, pretrained_model):
        super().__init__()
        self.pretrained_model = pretrained_model
        self.attention_weights = []
    
    def forward(self, x, return_attention=True):
        batch_size = x.size(0)
        
        x = self.pretrained_model.patch_embed(x)        
        x = x.flatten(2)
        x = x.transpose(1, 2)
        
        cls_tokens = self.pretrained_model.cls_token.expand(batch_size, -1, -1)
        x = torch.cat((cls_tokens, x), dim=1)
        
        if self.pretrained_model.pos_embed is not None:
            x = x + self.pretrained_model.pos_embed
        
        x = self.pretrained_model.dropout(x)
        
        self.attention_weights = []
        for layer in self.pretrained_model.transformer_encoder:
            attn_output, attn_weights = layer.attention(x, x, x)
            self.attention_weights.append(attn_weights)
            
            x = x + layer.dropout(attn_output)
            x = layer.norm1(x)
            
            mlp_output = layer.mlp(x)
            x = x + layer.dropout(mlp_output)
            x = layer.norm2(x)
        
        x = self.pretrained_model.norm(x)
        
        x_cls = x[:, 0]
        output = self.pretrained_model.head(x_cls)
        
        if return_attention:
            return output, self.attention_weights
        else:
            return output

def visualize_attention_maps(model, image, device,index, save_path='attention_maps'):
    img_size = 32
    patch_size = model.pretrained_model.patch_size
    n_patches = (img_size // patch_size) ** 2
    
    model.eval()
    with torch.no_grad():
        if isinstance(image, np.ndarray):
            image = torch.from_numpy(image).permute(2, 0, 1).unsqueeze(0).float()
        if image.dim() == 3:
            image = image.unsqueeze(0)
        image = image.to(device)
        
        _, attention_weights = model(image, return_attention=True)
    
    os.makedirs(save_path, exist_ok=True)
    
    last_attn = attention_weights[-1][0]  
    n_heads = last_attn.shape[0]
    
    fig, axes = plt.subplots(2, 4, figsize=(16, 8))
    axes = axes.flatten()
    
    for i in range(min(n_heads, 8)):
        cls_to_all = last_attn[i, 0, 1:]  # Exclude CLS->CLS attention
        cls_to_all = cls_to_all.reshape(img_size // patch_size, img_size // patch_size)
        
        axes[i].imshow(cls_to_all.cpu().numpy(), cmap='hot')
        axes[i].set_title(f'Head {i+1}')
        axes[i].axis('off')
    
    plt.tight_layout()
    plt.savefig(os.path.join(save_path, f'attention_heads_last_layer_{index}.png'))
    plt.close()
    
    avg_attn = last_attn.mean(0)[0, 1:]  
    avg_attn = avg_attn.reshape(img_size // patch_size, img_size // patch_size)
    
    plt.figure(figsize=(8, 8))
    plt.imshow(avg_attn.cpu().numpy(), cmap='hot')
    plt.title('Average Attention from CLS Token (Last Layer)')
    plt.colorbar()
    plt.savefig(os.path.join(save_path, f'average_attention_last_layer_{index}.png'))
    plt.close()
    
    n_layers = len(attention_weights)
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    axes = axes.flatten()
    
    for i in range(min(n_layers, 6)):
        layer_attn = attention_weights[i][0].mean(0)[0, 1:]  # [n_patches]
        layer_attn = layer_attn.reshape(img_size // patch_size, img_size // patch_size)
        
        axes[i].imshow(layer_attn.cpu().numpy(), cmap='hot')
        axes[i].set_title(f'Layer {i+1}')
        axes[i].axis('off')
    
    plt.tight_layout()
    plt.savefig(os.path.join(save_path, f'attention_across_layers_{index}.png'))
    plt.close()

In [12]:
def attention_rollout(attention_weights, device):
    attention_matrix = torch.eye(attention_weights[0].shape[-1]).to(device)
    for layer_attn in attention_weights:
        layer_attn = layer_attn.mean(1)
        layer_attn = layer_attn + torch.eye(layer_attn.shape[-1]).to(device)
        layer_attn = layer_attn / layer_attn.sum(dim=-1, keepdim=True)
        attention_matrix = torch.matmul(layer_attn, attention_matrix)
    
    return attention_matrix

def visualize_attention_rollout(model, image, device, index, save_path='attention_rollout'):
    img_size = 32
    patch_size = model.pretrained_model.patch_size
    model.eval()
    with torch.no_grad():
        if isinstance(image, np.ndarray):
            image = torch.from_numpy(image).permute(2, 0, 1).unsqueeze(0).float()
        if image.dim() == 3:
            image = image.unsqueeze(0)
        image = image.to(device)
        
        _, attention_weights = model(image, return_attention=True)
    
    rollout_matrix = attention_rollout(attention_weights, device)[0] 
    
    cls_to_all = rollout_matrix[0, 1:]  
    cls_to_all = cls_to_all.reshape(img_size // patch_size, img_size // patch_size)
    
    os.makedirs(save_path, exist_ok=True)
    
    plt.figure(figsize=(8, 8))
    plt.imshow(cls_to_all.cpu().numpy(), cmap='hot')
    plt.title('Attention Rollout from CLS Token')
    plt.colorbar()
    plt.savefig(os.path.join(save_path, f'attention_rollout_{index}.png'))
    plt.close()
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))
    
    original_img = image[0].cpu().numpy().transpose(1, 2, 0)
    mean = np.array([0.4914, 0.4822, 0.4465])
    std = np.array([0.2470, 0.2435, 0.2616])
    original_img = original_img * std + mean
    original_img = np.clip(original_img, 0, 1)
    
    ax1.imshow(original_img)
    ax1.set_title('Original Image')
    ax1.axis('off')
    
    ax2.imshow(cls_to_all.cpu().numpy(), cmap='hot')
    ax2.set_title('Attention Rollout')
    ax2.axis('off')
    
    plt.tight_layout()
    plt.savefig(os.path.join(save_path, f'attention_rollout_comparison_{index}.png'))
    plt.close()


In [24]:
def visualize_positional_embeddings(model,index, save_path='positional_embeddings'):
    pos_embed = model.pretrained_model.pos_embed[0] 
    sim_matrix = torch.matmul(pos_embed, pos_embed.transpose(0, 1))
    
    os.makedirs(save_path, exist_ok=True)
    
    plt.figure(figsize=(10, 8))
    plt.imshow(sim_matrix.detach().cpu().numpy(), cmap='viridis')
    plt.title('Positional Embedding Similarity Matrix')
    plt.xlabel('Position')
    plt.ylabel('Position')
    plt.colorbar()
    plt.tight_layout()
    plt.savefig(os.path.join(save_path, f'pos_embed_similarity_{index}.png'))
    plt.close()
    
    dist_matrix = torch.cdist(pos_embed, pos_embed, p=2)
    
    plt.figure(figsize=(10, 8))
    plt.imshow(dist_matrix.detach().cpu().numpy(), cmap='viridis')
    plt.title('Positional Embedding Distance Matrix')
    plt.xlabel('Position')
    plt.ylabel('Position')
    plt.colorbar()
    plt.tight_layout()
    plt.savefig(os.path.join(save_path, f'pos_embed_distance_{index}.png'))
    plt.close()


def run_all_visualizations(model_path, device, index):
    checkpoint = torch.load(model_path, map_location=device)

    best_config = checkpoint['config']
    pos_embedding_type = checkpoint['pos_embedding_type']
    print(pos_embedding_type)
  
    pretrained_model = VisionTransformerWithPosEmbed(
        img_size=32,
        patch_size=best_config['patch_size'],
        in_channels=3,
        num_classes=10,
        embed_dim=best_config['embed_dim'],
        depth=best_config['depth'],
        n_heads=best_config['n_heads'],
        mlp_dim=best_config['mlp_dim'],
        dropout=0.1,
        pos_embedding_type=pos_embedding_type
    ).to(device)
    
    pretrained_model.load_state_dict(checkpoint['model_state_dict'])
    vis_model = VisionTransformerForVisualization(pretrained_model)
    
    transform_test = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616)),
    ])
    
    test_dataset = torchvision.datasets.CIFAR10(
        root='./data', train=False, download=True, transform=transform_test)
    
    sample_idx = 100 
    sample_image, sample_label = test_dataset[sample_idx]
    
    print(f"Visualizing model with {pos_embedding_type} positional embedding...")
    print("Visualizing attention maps...")
    visualize_attention_maps(vis_model, sample_image, device, index, 'attention_maps')
    
    print("Visualizing attention rollout...")
    visualize_attention_rollout(vis_model, sample_image, device, index, 'attention_rollout')
    
    print("Visualizing positional embeddings...")
    visualize_positional_embeddings(vis_model, index, 'positional_embeddings')
    
    print("All visualizations completed!")


In [25]:
model_path = './saved_models/vit_cifar10_learned_1d_best.pth'
index = 'learned_1d'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
run_all_visualizations(model_path, device, index)

learned_1d
Visualizing model with learned_1d positional embedding...
Visualizing attention maps...
Visualizing attention rollout...
Visualizing positional embeddings...
All visualizations completed!


In [30]:
model_path = './saved_models/vit_cifar10_sinusoidal_best.pth'
index = 'sinusoidal'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
run_all_visualizations(model_path, device, index)

sinusoidal
Visualizing model with sinusoidal positional embedding...
Visualizing attention maps...
Visualizing attention rollout...
Visualizing positional embeddings...
All visualizations completed!


### Attention maps:
Image 1 shows attention patterns across six transformer layers, progressing from broad, diffuse attention in early layers (Layer 1) to more focused, object-specific attention in later layers (Layers 4-6). Image 2 displays different specializations across eight attention heads in the final layer, with some heads focusing on specific features (like edges or shapes) while others attend to broader patterns. Image 3 shows the average attention from the CLS token to all patches, highlighting the model's focus on salient image regions, with brighter areas (particularly at coordinates (4,4), (2,6), and (7,6)) receiving significantly more attention for classification.

### Attention rollout:
The attention rollout visualizations reveal how the ViT model integrates information across transformer layers when classifying images. In Image 1, we see a dog image (left) and its corresponding attention rollout map (right), which shows the model strongly focuses on the central region where the dog's face and body are located, with the brightest spots (white/yellow) highlighting the most important features for classification. Image 2 presents the same information as a heatmap with grid coordinates, clearly showing intense focus on patches (3,3) and (3,4) which correspond to the dog's main features, while maintaining moderate attention (red areas) to surrounding contextual regions. Attention rollout differs from regular attention maps as it accounts for information flow through all layers simultaneously by propagating attention in a principled way, providing a holistic view of how the model makes classification decisions.

### Positional Embeddings
Image 1 shows the L2 distance matrix between positional embeddings. The dark diagonal represents zero distance between identical positions, while increasing brightness (yellow) indicates greater distance between positions that are far apart. The periodic patterns and grid-like structures suggest the model has learned systematic spatial relationships where nearby positions have similar embeddings.
Image 2 displays the dot product similarity matrix of the same embeddings, essentially showing the inverse relationship. Higher similarity (green/yellow) appears between positions close to each other, while lower similarity (dark blue/purple) appears between distant positions. The model exhibits a gradual transition from highly similar adjacent positions to dissimilar distant ones.
Both visualizations confirm that the ViT has successfully learned meaningful 1D positional encodings that respect spatial locality - positions that are close together have similar embeddings, while those far apart have more distinct representations. This spatial awareness is crucial for the model to understand image structure despite the token sequence being flattened.RetryClaude can make mistakes. Please double-check responses. 3.7 Sonnet
